In [2]:
###Importing the libraries for preprocessing
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
import warnings
warnings.filterwarnings("ignore")

In [21]:
##Reading the file 
data = pd.read_json(r"C:\Users\PSALISHOL\Documents\My Projects\Car Prediction\data\raw\Acura_model.json",orient="columns")
data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[]
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[]
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[]
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[]
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[]


### Cleaning DrivetrainType

In [20]:
data["drivetrain_type"].value_counts()

All-wheel Drive      4072
Front-wheel Drive    3741
AWD                   451
FWD                   315
Four-wheel Drive       28
–                      25
4WD                    10
Rear-wheel Drive        6
All Wheel Drive         2
RWD                     1
Name: drivetrain_type, dtype: int64

#### According to above, some of the transmission has All wheel Drive represented as 'All-wheel Drive' and 'AWD', Four wheel Drive as 'Forward-wheel Drive' and '4WD' etc. 
#### we'll now be making some transformation. The transformation include turning;
- #### All-wheel Drive to AWD
- #### Front-wheel Drive FWD
- #### Four-wheel Drive to 4WD
- #### Rear-wheel Drive to RWD
- #### - to NaN


In [4]:
def clean_drivetrain(new_data,drop=True):
    
        new_data["Drivetrain"] = new_data["drivetrain_type"].apply(lambda info: info.replace("All-wheel Drive", "AWD").replace(
            "Front-wheel Drive", "FWD").replace("Four-wheel Drive", "4WD").replace("Rear-wheel Drive", "RWD").replace("All Wheel Drive", "AWD"))
        i =0
        # Replacing "-" with nan
        for feature in new_data["Drivetrain"]:
            if feature == "–":
                new_data.loc[i, "Drivetrain"] = np.nan
            i +=1
        # Dropping the columns
        if drop is True:
            new_data = new_data.drop(
                "drivetrain_type", axis="columns")

        return new_data
    
new_data = clean_drivetrain(data,drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Drivetrain
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],FWD
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],AWD
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],AWD
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],FWD
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],FWD


In [54]:
# Rechecking
new_data["Drivetrain"].value_counts()

AWD    4525
FWD    4056
4WD      38
RWD       7
Name: Drivetrain, dtype: int64

### Cleaning Product_type

In [5]:
data["product_type"].value_counts()

Used                       5287
New                        2126
Acura Certified            1236
Mercedes-Benz Certified       1
Volvo Certified               1
Name: product_type, dtype: int64

#### Transformation Done include Turning model Certified to Certified.

In [6]:
def mk_pd_type(new_data, drop=True):
    i = 0
    new_data["Type"] = np.nan
    for feature in new_data["product_type"]:
        if feature in ["Used", "New"]:
            new_data.loc[i, "Type"] = feature
        else:
            new_data.loc[i, "Type"] = "Certified"
        i += 1

    if drop is True:
        new_data = new_data.drop("product_type", axis=1)
    return new_data

new_data = mk_pd_type(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Type
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],Used
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],Used
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Used
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Certified
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],Certified


In [33]:
new_data["Type"].value_counts()

Used         5287
New          2126
Certified    1238
Name: Type, dtype: int64

### Cleaning Transmission Type

In [32]:
data["transmission"].value_counts()

10-Speed Automatic                   3071
9-Speed Automatic                    2028
6-Speed Automatic                    1168
8-Speed Automatic with Auto-Shift    1006
5-Speed Automatic                     591
Automatic                             558
7-Speed Automatic with Auto-Shift      40
9-Speed Automatic with Auto-Shift      22
–                                      19
6-Speed Manual                         17
2.0L I4 16V GDI DOHC Turbo             15
4-Speed Automatic                      12
5-Speed Manual                         12
automatic                              11
I4                                     11
8-Speed Automatic                       8
3.5L V6 24V GDI SOHC                    7
Automatic CVT                           6
2.0L                                    5
8-Speed Dual-Clutch                     5
5-SPEED A/T                             4
2.7L V6 24V SOHC                        3
8-SPEED A/T                             3
7-Speed Automatic                 

#### For transmission type, we'll be replacing all instance with A/T, Automatic, etc with Automatic, and those with Manual with Manual, and Cvt with CVT. and otherwise with NaN
#### For intance, 10-Speed A/T will be turned to Automatic, 6-Speed Shfitable Automatic to Automatic, 5-Speed Manual to Manual. etc

In [8]:
def clean_transmission(new_data, drop=True):
    # using the same thing for the above and all in the same accord
    new_data["Transmission"] = np.nan
    for i in range(len(new_data["transmission"])):
        if new_data["transmission"][i] in ["10-Speed Automatic", "10-SPEED A/T", "10 Cylinder", "9-Speed Automatic", "9-Speed Automatic with Auto-Shift",
                                                "8-Speed Automatic with Auto-Shift", "8-Speed Automatic", "8-SPEED A/T", "7-Speed Automatic with Auto-Shift", "7-Speed Automatic", "7-SPEED A/T",
                                                "automatic", "6-Speed Automatic", "5-Speed Automatic", "Automatic", "4-Speed Automatic", "5-SPEED A/T", "6-SPEED A/T", "Automatic 5-Speed", "AUTOMATIC",
                                                "A", "Auto, 9-Spd w/SprtShft", "Auto, 5-Spd w/SportShift", "6-Speed Shiftable Automatic", "5-Speed Automatic with Overdrive"]:

            new_data.loc[i, "Transmission"] = "Automatic"

        elif new_data["transmission"][i] in ["6-Speed Manual", "5-Speed Manual", "Manual", "8-Speed Auto-Shift Manual w/OD"]:
            new_data.loc[i, "Transmission"] = "Manual"

        elif new_data["transmission"][i] in ["Automatic CVT"]:
            new_data.loc[i, "Transmission"] = "Continous Variable"

        elif new_data["transmission"][i] in ["8-Speed Dual-Clutch", "8-Speed Double Clutch", ""]:
            new_data.loc[i, "Transmission"] = "Double Clutch"

    if drop is True:
        new_data = new_data.drop("transmission", axis=1)
    return new_data

new_data = clean_transmission(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Transmission
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],Automatic
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],Automatic
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Automatic
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Automatic
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],Automatic


In [59]:
new_data["Transmission"].value_counts()

Automatic             8541
Manual                  33
Double Clutch            6
Continous Variable       6
Name: Transmission, dtype: int64

### Extracting Engine size

In [60]:
data["engine_type"].value_counts()

3.5L V6 24V GDI SOHC                                                2772
2.0L I4 16V GDI DOHC Turbo                                          2526
2.4L I4 16V GDI DOHC                                                1006
3.5L V6 24V MPFI SOHC                                                711
3.7L V6 24V MPFI SOHC                                                367
2.4L I4 16V MPFI DOHC                                                191
Intercooled Turbo Premium Unleaded I-4 2.0 L/122                     183
Premium Unleaded V-6 3.5 L/212                                       154
3.2L V6 24V MPFI SOHC                                                 79
2.0L I4 16V MPFI SOHC                                                 71
2.3L I4 16V MPFI DOHC Turbo                                           69
Premium Unleaded I-4 2.4 L/144                                        69
3.0L V6 24V GDI SOHC Turbo                                            65
V6 Cylinder Engine                                 

#### For extracting engine size; in Vehicles, the engine size is mostly represented in Num with L followed by it, indicating the size of the vehicle. in this, the engine size is the first in the engine info, which makes it easier to extract. 

In [10]:
def extract_eng_size(new_data,drop=True):

    i = 0
    new_data["Engine size"] = np.nan
    for feature in new_data["engine_type"]:
        try:
            new_data.loc[i, "Engine size"] = float(
                feature.split(" ")[0].translate({ord(s): None for s in ["L"]}))
        except Exception:
            new_data.loc[i, "Engine size"] = np.nan
        i += 1

    # Dropping the engine_type
    if drop is True:
        new_data = new_data.drop("engine_type", axis="columns")
    return new_data

new_data = extract_eng_size(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Engine size
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],1.5
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],2.0
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],2.0
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],2.4
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],2.0


### Cleaning mpg

In [62]:
data["mpg"].value_counts()

21–27       758
19–26       620
19–25       552
21–26       518
22–28       440
18–26       435
19–27       428
22–31       372
24–34       313
20–27       310
20–29       309
21–29       291
20–28       283
23–33       246
16–21       238
22–27       212
24–35       209
20–31       194
25–35       188
18–27       159
21–34       126
0–26        109
0–27         93
20–30        92
22–30        90
21–30        81
25–36        80
17–22        58
20–32        56
23–32        56
15–20        55
19–28        50
0–31         49
0–25         43
21–31        43
0–29         42
19–24        40
17–23        36
0–33         29
26–27        27
0–28         24
21–22        23
0–34         22
0–35         18
0–30         18
17–25        16
19–23        15
19–29        13
28–29        10
18–24         9
25–34         8
16–23         6
0–0           6
27–33         5
0–32          5
39–38         4
0–22          4
0–21          3
16–22         3
0–36          3
24–33         3
29–30         3
0–20    

#### Operation performed include removing hypen and aggregating the varying score.

In [12]:
def clean_mpg(new_data, drop=True):
    # Making a new variable for Avg mpg
    new_data["Avg mpg"] = np.nan
    for i in range(len(new_data["mpg"])):
        if new_data["mpg"][i] is not None and len(new_data["mpg"][i].split("–")) == 1:
            new_data.loc[i, "Avg mpg"] = int(
                new_data["mpg"][i].split("–")[0])
        else:
            try:
                new_data.loc[i, "Avg mpg"] = (int(new_data["mpg"][i].split("–")[
                                                    0])+int(new_data["mpg"][i].split("–")[1]))/2
            except Exception:
                new_data.loc[i, "Avg mpg"] = np.nan

    if drop is True:
        new_data = new_data.drop("mpg", axis="columns")

    return new_data

new_data = clean_mpg(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Avg mpg
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],34.5
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],23.5
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],23.5
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],28.0
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],24.5


In [64]:
new_data["Avg mpg"].value_counts()

24.0       1054
22.0        987
23.5        878
25.0        823
22.5        779
24.5        521
23.0        428
26.5        399
29.0        313
25.5        275
28.0        248
18.5        239
29.5        220
30.0        193
26.0        190
27.5        184
13.0        109
13.5         93
30.5         81
17.5         73
21.5         66
19.5         64
15.5         50
12.5         44
14.5         42
21.0         41
20.0         39
16.5         29
14.0         24
17.0         22
15.0         18
28.5         13
0.0           6
16.0          5
10.5          4
38.5          4
19.0          4
11.0          4
12.0          3
18.0          3
10.0          3
27.0          2
20.5          2
34.5          1
9.0           1
31.0          1
10509.0       1
11.5          1
9.5           1
42.0          1
Name: Avg mpg, dtype: int64

### Extracting seller Location

In [65]:
data["seller_info"]

0                1886 S Elmhurst Rd Mt Prospect, IL 60056
1                          20420 WA-99 Lynnwood, WA 98036
2                          17315 WA-99 Lynnwood, WA 98037
3                       4051 W Plano Pkwy Plano, TX 75093
4                   1125 Gulf Fwy S League City, TX 77573
5                    2275 Aurora Ave Naperville, IL 60540
6                1500 E State Hwy 114 Grapevine, TX 76051
7                      1100 S Madera Ave Madera, CA 93637
8             1099 W Higgins Rd Hoffman Estates, IL 60169
9                      4211 S Limit Ave Sedalia, MO 65301
10                        ONLINE ONLY Knoxville, TN 37923
11                 14080 Brookpark Rd Cleveland, OH 44135
12            11777 Tesson Ferry Rd Saint Louis, MO 63128
13             11225 Atlantic Blvd Jacksonville, FL 32225
14                    8545 Leesburg Pike Vienna, VA 22182
15                   4460 Tradition Trail Plano, TX 75093
16              2909 Pacific Coast Hwy Torrance, CA 90505
17           6

#### Extracting Seller State, City and Zip 

In [14]:
def extract_loc(new_data, drop=True):
    
    new_data["State"] = new_data["seller_info"].apply(
        lambda info: info.split(",")[1].strip().split(" ")[0])

    # Extracting seller Zip
    i = 0
    new_data["Zip"] = np.nan
    for feature in new_data["seller_info"]:
        if len(feature.split(",")[-1].strip().split(" ")) == 2:
            new_data.loc[i, "Zip"] = feature.split(
                ",")[1].strip().split(" ")[-1]
        else:
            new_data.loc[i, "Zip"] = np.nan
        i += 1

    # Extracting Seller city
    new_data["City"] = new_data["seller_info"].apply(
        lambda info: info.split(",")[0].split(" ")[-1])

    if drop is True:
        new_data = new_data.drop("seller_info", axis=1)

    return new_data

new_data = extract_loc(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,State,Zip,City
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],IL,60056,Prospect
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],WA,98036,Lynnwood
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],WA,98037,Lynnwood
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],TX,75093,Plano
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],TX,77573,City


In [67]:
new_data["State"].value_counts()

CA            892
FL            862
TX            725
IL            528
NY            513
NJ            471
PA            402
OH            369
MD            333
VA            329
MA            264
GA            257
NC            232
MO            199
CO            197
CT            186
IN            182
MN            144
WA            138
AZ            134
TN            127
WI             99
KS             98
AL             83
SC             72
HI             72
OR             69
NH             61
MI             60
NV             59
UT             56
KY             55
OK             51
IA             45
DE             38
MS             34
NE             33
LA             31
RI             29
AR             25
Colorado       16
Tulsa          15
NM             13
ID             11
SD              8
ND              6
AK              6
West            4
WV              3
WY              2
NW              2
ME              2
San             1
Building        1
Matteson        1
Michigan  

### Extracting the vehicle model and make

In [68]:
data["vehicle_made"].value_counts()

Acura RDX Technology Package                        828
Acura RDX A-Spec                                    748
Acura MDX 3.5L w/Technology Package                 654
Acura MDX 3.5L                                      549
Acura RDX Base                                      541
Acura TLX Technology                                376
Acura RDX Advance Package                           293
Acura TLX FWD                                       230
Acura MDX Technology                                224
Acura ILX Premium Package                           188
Acura TLX V6 w/Technology Package                   174
Acura TLX Advance                                   166
Acura MDX 3.5L w/Advance Package                    163
Acura TLX A-Spec                                    163
Acura MDX 3.7L Technology                           139
Acura ILX Base                                      134
Acura TLX V6                                        133
Acura MDX Advance                               

In [16]:
def extract_model(new_data, drop=False):
        
        i = 0
        # Making a new feature for Car make
        new_data["Model"] = np.nan
        new_data["Vehicle Make"] = np.nan
        for feature in new_data["vehicle_made"]:
            new_data.loc[i, "Vehicle Make"] = feature.split(" ")[
                0].strip()
            new_data.loc[i, "Model"] = feature.split(" ")[1].strip()
            i += 1

        if drop is True:
            new_data = new_data.drop("vehicle_made", axis="columns")

        return new_data
new_data = extract_model(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Model,Vehicle Make
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],Hardtop,MINI
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],RDX,Acura
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],RDX,Acura
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],TLX,Acura
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],RDX,Acura


### Extracting Features

In [70]:
data["vehicle_features"]

0       [Heated Seats, Keyless Start, Bluetooth, Alloy...
1       [Adaptive Cruise Control, Navigation System, P...
2       [Adaptive Cruise Control, Heated Seats, Naviga...
3       [Adaptive Cruise Control, Heated Seats, Naviga...
4       [Adaptive Cruise Control, Keyless Start, Navig...
5       [Adaptive Cruise Control, Heated Seats, Keyles...
6       [Adaptive Cruise Control, Heated Seats, Power ...
7       [Adaptive Cruise Control, Heated Seats, Naviga...
8       [Adaptive Cruise Control, Heated Seats, Naviga...
9       [Adaptive Cruise Control, Navigation System, P...
10      [Adaptive Cruise Control, Heated Seats, Keyles...
11      [Adaptive Cruise Control, Heated Seats, Naviga...
12      [Adaptive Cruise Control, Heated Seats, Naviga...
13      [Adaptive Cruise Control, Heated Seats, Naviga...
14      [Adaptive Cruise Control, Heated Seats, Heated...
15      [Heated Seats, Power Liftgate, Bluetooth, Home...
16      [Adaptive Cruise Control, Heated Seats, Keyles...
17      [Adapt

#### Putting all the vehicle features together and making each of them a new variable

In [18]:
def extract_vinfo(new_data, drop=True):
    # Collecting all the car features and making a new faeture out of them
    all_vfea = []
    for i in range(0, len(new_data["vehicle_features"])):
        d_point = new_data["vehicle_features"][i]
        for feature in d_point:
            if feature not in all_vfea:
                all_vfea.append(feature)

    # making the new featue
    for feature in all_vfea:
        # Creating an empty column with each vehiclefeature as Variable
        new_data[feature] = np.nan
        for i in range(0, len(new_data["vehicle_features"])):
            if feature in new_data["vehicle_features"][i]:
                new_data.loc[i, feature] = "Yes"
            else:
                new_data.loc[i, feature] = "No"

    # Dropping Vehicle features
    if drop is True:
        new_data = new_data.drop(
            ["vehicle_features", "files"], axis=1)

    return new_data

new_data = extract_vinfo(new_data=data, drop=False)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files,Heated Seats,Keyless Start,Bluetooth,Alloy Wheels,Sunroof/Moonroof,Brake Assist,Stability Control,Leather Seats,Adaptive Cruise Control,Navigation System,Power Liftgate,Premium Sound System,Memory Seat,Apple CarPlay/Android Auto,HomeLink,Backup Camera,Lane Departure Warning,Blind Spot Monitor,USB Port,Automatic Emergency Braking,Rear Cross Traffic Alert,Third Row Seating,Remote Start,LED Headlights,Heated Steering Wheel,Tow Hitch,Cooled Seats,Rear Seat Entertainment
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[],Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[],No,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[],Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[],No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No


### Cleaning fuel

In [20]:
def clean_fuel(new_data):
    i = 0
    for inf in new_data["fueltype"]:
        if inf != "Gasoline":
            new_data.loc[i, "fueltype"] = np.nan
        else:
            new_data.loc[i, "fueltype"] = "Gasoline"
        i += 1

    return new_data
new_data = clean_fuel(new_data=data)
new_data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files
0,Used,2015,MINI Hardtop Cooper,72528,12980,Front-wheel Drive,Gasoline,29–40,6-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,"1886 S Elmhurst Rd Mt Prospect, IL 60056","[Heated Seats, Keyless Start, Bluetooth, Alloy...",[]
1,Used,2019,Acura RDX A-Spec,36182,41998,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"20420 WA-99 Lynnwood, WA 98036","[Adaptive Cruise Control, Navigation System, P...",[]
2,Used,2019,Acura RDX A-Spec,12775,44499,All-wheel Drive,Gasoline,21–26,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"17315 WA-99 Lynnwood, WA 98037","[Adaptive Cruise Control, Heated Seats, Naviga...",[]
3,Acura Certified,2020,Acura TLX Technology,31175,31140,Front-wheel Drive,Gasoline,23–33,8-Speed Automatic with Auto-Shift,2.4L I4 16V GDI DOHC,"4051 W Plano Pkwy Plano, TX 75093","[Adaptive Cruise Control, Heated Seats, Naviga...",[]
4,Acura Certified,2019,Acura RDX A-Spec,75689,34495,Front-wheel Drive,Gasoline,22–27,10-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1125 Gulf Fwy S League City, TX 77573","[Adaptive Cruise Control, Keyless Start, Navig...",[]


### Putting it all together

In [22]:
def clean_data(dataf,drop_cols=True):

    if drop_cols is False:
        dataf = clean_fuel(data=dataf)
        dataf = extract_loc(new_data=dataf, drop=False)
        dataf = clean_drivetrain(new_data=dataf,drop=False)
        dataf = clean_transmission(new_data=dataf,drop=False)
        dataf = extract_eng_size(new_data=dataf,drop=False)
        dataf = clean_mpg(new_data=dataf,drop=False)
        dataf = extract_model(new_data=dataf,drop=False)
        dataf = extract_vinfo(new_data=dataf,drop=False)
        dataf = mk_pd_type(new_data=dataf,drop=False)

    else:
        dataf = clean_fuel(new_data=dataf)
        dataf = extract_loc(new_data=dataf, drop=True)
        dataf = clean_drivetrain(new_data=dataf,drop=True)
        dataf = clean_transmission(new_data=dataf,drop=True)
        dataf = extract_eng_size(new_data=dataf,drop=True)
        dataf = clean_mpg(new_data=dataf,drop=True)
        dataf = extract_model(new_data=dataf,drop=True)
        dataf = extract_vinfo(new_data=dataf,drop=True)
        dataf = mk_pd_type(new_data=dataf,drop=True)

    return dataf

In [23]:
### Cleaning the data
new_data = clean_data(dataf=data, drop_cols=True)
new_data.head()

,year,mileage,price,fueltype,State,Zip,City,Drivetrain,Transmission,Engine size,Avg mpg,Model,Vehicle Make,Heated Seats,Keyless Start,Bluetooth,Alloy Wheels,Sunroof/Moonroof,Brake Assist,Stability Control,Leather Seats,Adaptive Cruise Control,Navigation System,Power Liftgate,Premium Sound System,Memory Seat,Apple CarPlay/Android Auto,HomeLink,Backup Camera,Lane Departure Warning,Blind Spot Monitor,USB Port,Automatic Emergency Braking,Rear Cross Traffic Alert,Third Row Seating,Remote Start,LED Headlights,Heated Steering Wheel,Tow Hitch,Cooled Seats,Rear Seat Entertainment,Type
0,2015,72528,12980,Gasoline,IL,60056,Prospect,FWD,Automatic,1.5,34.5,Hardtop,MINI,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Used
1,2019,36182,41998,Gasoline,WA,98036,Lynnwood,AWD,Automatic,2.0,23.5,RDX,Acura,No,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Used
2,2019,12775,44499,Gasoline,WA,98037,Lynnwood,AWD,Automatic,2.0,23.5,RDX,Acura,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,Used
3,2020,31175,31140,Gasoline,TX,75093,Plano,FWD,Automatic,2.4,28.0,TLX,Acura,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,Certified
4,2019,75689,34495,Gasoline,TX,77573,City,FWD,Automatic,2.0,24.5,RDX,Acura,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Certified


In [2]:
import pandas as pd
data = pd.read_json(r"C:\Users\PSALISHOL\Documents\My Projects\Car Prediction\data\raw\BMW.json",orient="columns")
data.head()

,product_type,year,vehicle_made,mileage,price,drivetrain_type,fueltype,mpg,transmission,engine_type,seller_info,vehicle_features,files
0,Used,2019,Acura MDX 3.5L w/Technology Package,11323,43995,All-wheel Drive,Gasoline,19–26,9-Speed Automatic,3.5L V6 24V GDI SOHC,"1924 N Paulina St Chicago, IL 60622","[Adaptive Cruise Control, Heated Seats, Keyles...",[]
1,BMW Certified,2019,BMW X5 xDrive40i,14613,60495,All-wheel Drive,Gasoline,20–26,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,"1675 Howard St San Francisco, CA 94103","[Heated Seats, Heated Steering Wheel, Navigati...",[]
2,Used,2018,BMW X2 xDrive28i,18066,32995,All-wheel Drive,Gasoline,21–31,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,"1501 Alpharetta Hwy Alpharetta, GA 30076","[Heated Seats, Heated Steering Wheel, Navigati...",[]
3,Used,2015,BMW X5 xDrive35i,60021,23990,All-wheel Drive,Gasoline,18–27,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,"300 Great Neck Road Great Neck, NY 11021","[Heated Seats, Keyless Start, Navigation Syste...",[]
4,Used,2018,BMW 440 i xDrive,34787,39450,All-wheel Drive,Gasoline,21–31,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,"1020 U.S. 1 Avenel, NJ 07001","[Heated Seats, Navigation System, Bluetooth, P...",[]


In [4]:
data["fueltype"].value_counts()

Gasoline                               8583
8-Speed Automatic                       380
1-Speed Automatic                       100
Diesel                                   81
6-Speed Automatic                        59
Hybrid                                   19
Automatic                                15
Electric                                  5
8-Speed Automatic with Steptronic         2
5-Speed Manual                            2
–                                         1
Electronically Controlled Automatic       1
Name: fueltype, dtype: int64